In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Run RAG Pipelines in BigQuery with BQML and Vector Search

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retrieval-augmented-generation/rag_with_bigquery.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fuse-cases%2Fretrieval-augmented-generation%2Frag_with_bigquery.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/use-cases/retrieval-augmented-generation/rag_with_bigquery.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retrieval-augmented-generation/rag_with_bigquery.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/bigquery/v1/32px.svg" alt="BigQuery Studio logo"><br> Open in BigQuery Studio
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retrieval-augmented-generation/rag_with_bigquery.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retrieval-augmented-generation/rag_with_bigquery.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retrieval-augmented-generation/rag_with_bigquery.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retrieval-augmented-generation/rag_with_bigquery.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retrieval-augmented-generation/rag_with_bigquery.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retrieval-augmented-generation/rag_with_bigquery.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>            

| | |
|-|-|
| Author(s) | [Jeff Nelson](https://github.com/jeffonelson/), Eric Hao |

## Overview

This notebook demonstrates a basic end-to-end retrieval-augmented generation (RAG) pipeline using [BigQuery](https://cloud.google.com/bigquery/) and [BigQuery ML](https://cloud.google.com/bigquery/docs/bqml-introduction) functions. To do so, we:

* Complete setup steps to download sample data and access [Vertex AI](https://cloud.google.com/vertex-ai) from BigQuery
* Generate [object table](https://cloud.google.com/bigquery/docs/object-table-introduction) to access unstructured PDFs that reside in [Cloud Storage](https://cloud.google.com/storage)
* Create a remote model, so BigQuery can call [Document AI](https://cloud.google.com/document-ai) to parse the PDF inputs
* Parse response from Document AI into chunks and metadata, then generate vector embeddings for the chunks
* Run a [vector search](https://cloud.google.com/bigquery/docs/vector-search) against embeddings in BigQuery, return relevant chunks, and summarize them with Gemini

## How to open this notebook in BigQuery Studio

This notebook was written to be compatible for use within BigQuery Studio. To open this notebook in BigQuery, click to [Run in Colab Enterprise](https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fuse-cases%2Fretrieval-augmented-generation%2Frag_with_bigquery.ipynb). This will open a new window in the Cloud Console and prompt you to confirm import. Then, navigate to BigQuery, where you will find the notebook available in the Explorer pane under Notebooks.

## About the dataset

This example uses [the Federal Reserve's 2023 Survey of Consumer Finances](https://www.federalreserve.gov/publications/files/scf23.pdf) (SCF) report. The document contains information around US family income, net worth, credit use, and other common household financial indicators.

## Services and Costs

This tutorial uses the following Google Cloud data analytics and ML services, they are billable components of Google Cloud:

* BigQuery & BigQuery ML [(pricing)](https://cloud.google.com/bigquery/pricing)
* Vertex AI Generative AI models [(pricing)](https://cloud.google.com/vertex-ai/generative-ai/pricing)
* Document AI [(pricing)](https://cloud.google.com/document-ai/pricing)
* Cloud Storage [(pricing)](https://cloud.google.com/storage/pricing)

Use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

# Setup Steps to access Vertex AI models from BigQuery and enable APIs

### Install Document AI SDK

In [1]:
%pip install --quiet google-cloud-documentai==2.31.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 5.3 MB/s eta 0:00:00


### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()
    print("Authenticated")

### Define your Google Cloud project

In [2]:
PROJECT_ID = "gcp-strategicaccoun-prd-73127"  # @param {type: "string"}
PROJECT_NUMBER = "730191379179"  # @param {type: "string"}

### Enable Data Table Display

This makes it easier to visualize tabular data within a Notebook environment later on.

In [3]:
%load_ext google.colab.data_table

### Create a new dataset in BigQuery

This will house any tables created throughout this notebook.

In [3]:
!bq mk --location=us --dataset --project_id={PROJECT_ID} docai_demo

BigQuery error in mk operation: Dataset 'gcp-strategicaccoun-prd-73127:docai_demo' already exists.


### Create a Cloud resource connection

[Cloud resource connections](https://cloud.google.com/bigquery/docs/create-cloud-resource-connection) enable BigQuery to access other Cloud services, like Cloud Storage and Vertex AI.

In [4]:
!bq mk --connection --connection_type=CLOUD_RESOURCE --location=us --project_id={PROJECT_ID} "demo_conn"
!bq show --location=us --connection --project_id={PROJECT_ID} "demo_conn"

BigQuery error in mk operation: Already Exists: Connection
projects/730191379179/locations/us/connections/demo_conn
Connection gcp-strategicaccoun-prd-73127.us.demo_conn

            name              friendlyName   description    Last modified         type        hasCredential                                            properties                                            
 --------------------------- -------------- ------------- ----------------- ---------------- --------------- ----------------------------------------------------------------------------------------------- 
  730191379179.us.demo_conn                                04 Apr 17:40:36   CLOUD_RESOURCE   False           {"serviceAccountId": "bqcx-730191379179-pt2c@gcp-sa-bigquery-condel.iam.gserviceaccount.com"}  



### Add permissions to Cloud resource connection service account

The Cloud resource connection is associated with a service account. The following cell enables the service account to access services like Document AI, Cloud Storage, and Vertex AI.

**Note:** Copy the service account ID from the prior cell and input it below. It will look like `your-copied-service-account@gcp-sa-bigquery-condel.iam.gserviceaccount.com`.

In [6]:
connection_service_account = "bqcx-730191379179-pt2c@gcp-sa-bigquery-condel.iam.gserviceaccount.com"  # @param {type: "string"}
connection_member = f"serviceAccount:{connection_service_account}"


!gcloud projects add-iam-policy-binding {PROJECT_ID} --member={connection_member} --role='roles/documentai.viewer' --condition=None --quiet
!gcloud projects add-iam-policy-binding {PROJECT_ID} --member={connection_member} --role='roles/storage.objectViewer' --condition=None --quiet
!gcloud projects add-iam-policy-binding {PROJECT_ID} --member={connection_member} --role='roles/storage.objectUser' --condition=None --quiet
!gcloud projects add-iam-policy-binding {PROJECT_ID} --member={connection_member} --role='roles/aiplatform.user' --condition=None --quiet

Updated IAM policy for project [gcp-strategicaccoun-prd-73127].
bindings:
- members:
  - serviceAccount:pulse-service-account@gcp-strategicaccoun-prd-73127.iam.gserviceaccount.com
  role: roles/aiplatform.admin
- members:
  - serviceAccount:service-730191379179@gcp-sa-vertex-nb.iam.gserviceaccount.com
  - serviceAccount:vertex-api@gcp-strategicaccoun-prd-73127.iam.gserviceaccount.com
  role: roles/aiplatform.colabServiceAgent
- members:
  - serviceAccount:service-730191379179@gcp-sa-aiplatform-vm.iam.gserviceaccount.com
  role: roles/aiplatform.notebookServiceAgent
- members:
  - serviceAccount:service-730191379179@gcp-sa-aiplatform.iam.gserviceaccount.com
  - serviceAccount:vertex-api@gcp-strategicaccoun-prd-73127.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - deleted:serviceAccount:bqcx-730191379179-1edr@gcp-sa-bigquery-condel.iam.gserviceaccount.com?uid=101552285927150338410
  - deleted:serviceAccount:bqcx-730191379179-4huz@gcp-sa-bigquery-condel.iam.gs

### Download the sample PDF used for this notebook and store it in a new Cloud Storage bucket

In [7]:
import random

# Create a unique Cloud Storage bucket name
bucket_name = f"{PROJECT_ID}-{random.randint(10000, 99999)}"

# Create the bucket
!gsutil mb -l US -p {PROJECT_ID} gs://{bucket_name}

# Download the PDF sample
!wget scf23.pdf "https://www.federalreserve.gov/publications/files/scf23.pdf"

# Upload the PDF sample to the newly created Cloud Storage bucket
!gsutil cp scf23.pdf gs://{bucket_name}/

# Print confirmation
print(f"PDF uploaded to gs://{bucket_name}/scf23.pdf")

Creating gs://gcp-strategicaccoun-prd-73127-47939/...
--2025-04-02 16:52:40--  http://scf23.pdf/
Resolving scf23.pdf (scf23.pdf)... failed: Name or service not known.
wget: unable to resolve host address ‘scf23.pdf’
--2025-04-02 16:52:40--  https://www.federalreserve.gov/publications/files/scf23.pdf
Resolving www.federalreserve.gov (www.federalreserve.gov)... 104.18.33.249, 172.64.154.7, 2606:4700:4400::ac40:9a07, ...
Connecting to www.federalreserve.gov (www.federalreserve.gov)|104.18.33.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10136125 (9.7M) [application/pdf]
Saving to: ‘scf23.pdf’

scf23.pdf           100%[===================>]   9.67M  --.-KB/s    in 0.06s   

2025-04-02 16:52:40 (165 MB/s) - ‘scf23.pdf’ saved [10136125/10136125]

FINISHED --2025-04-02 16:52:40--
Total wall clock time: 0.2s
Downloaded: 1 files, 9.7M in 0.06s (165 MB/s)
Copying file://scf23.pdf [Content-Type=application/pdf]...
-
Operation completed over 1 objects/9.7 MiB.      

## Create an object table

An object table allows BigQuery to read unstructured data in Google Cloud Storage. This uses the BigQuery Python client library to continue using the `bucket_name` variable.

In [45]:
from google.cloud import bigquery

client = bigquery.Client(project=PROJECT_ID)

query = f"""
CREATE OR REPLACE EXTERNAL TABLE `docai_demo.object_table`
WITH CONNECTION `us.demo_conn`  -- Replace with your connection ID
OPTIONS (
  uris = ['gs://cisco_factset/Earnings_Transcripts/Competitor Earnings Q1 2025/*.pdf'],
  object_metadata = 'DIRECTORY'
);
"""

query_job = client.query(query)  # API request
query_job.result()  # Waits for the query to complete

print("External table docai_demo.object_table created or replaced successfully.")

External table docai_demo.object_table created or replaced successfully.


In [24]:
from google.cloud import bigquery

client = bigquery.Client(project=PROJECT_ID)

query = f"""
INSERT INTO `docai_demo.object_table`
WITH CONNECTION `us.demo_conn`  -- Replace with your connection ID
OPTIONS (
  uris = ['gs://cisco_factset/Earnings_Transcripts/Competitor Earnings From Jul 2023/PANW*.pdf'],
  object_metadata = 'DIRECTORY'
);
"""

query_job = client.query(query)  # API request
query_job.result()  # Waits for the query to complete

print("External table docai_demo.object_table created or replaced successfully.")

BadRequest: 400 Syntax error: Expected keyword AS but got identifier `us.demo_conn` at [3:17]; reason: invalidQuery, location: query, message: Syntax error: Expected keyword AS but got identifier `us.demo_conn` at [3:17]

Location: US
Job ID: b30352f7-041a-4216-8b8a-ffbacddfdafe


In [16]:
bucket_name = ""

In [17]:
from google.cloud import storage
def list_blobs(bucket_name, prefix=""):
    """Lists all the blobs in the bucket."""
    #bucket_name = "your-bucket-name"

    storage_client = storage.Client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    blobs = storage_client.list_blobs(bucket_name)

    # Note: The call returns a response only when the iterator is consumed.
    for blob in blobs:
        print(blob.name)


In [18]:
list_blobs(bucket_name="cisco_factset",
           prefix="cisco_factset/Earnings_Transcripts/Competitor Earnings From Jul 2023"
           )

Earnings_Transcripts/
Earnings_Transcripts/Competitor Earnings From Jul 2023/000977 - 浪潮信息2024年10月17日投资者关系活动记录表 - 18-Oct-24.pdf
Earnings_Transcripts/Competitor Earnings From Jul 2023/000977 - 浪潮信息2024年5月11日14日投资者关系活动记录表 - 15-May-24.pdf
Earnings_Transcripts/Competitor Earnings From Jul 2023/000977 - 浪潮信息2024年5月15日投资者关系活动记录表 - 15-May-24.pdf
Earnings_Transcripts/Competitor Earnings From Jul 2023/000977 - 浪潮信息2024年6月13日投资者关系活动记录表 - 14-Jun-24.pdf
Earnings_Transcripts/Competitor Earnings From Jul 2023/300628 - Event Yealink Network Technology Co Ltd 2023 Annual Report Performance Briefing and Investor Online Collective Reception Day 16May2024 1400 AsiaShanghai Time GMT0800 - 15-May-24.pdf
Earnings_Transcripts/Competitor Earnings From Jul 2023/300628 - Event Yealink Network Technology Co Ltd Online Performance Meeting of 1st Half of 2023 27Sep2023 1530 AsiaShanghai Time GMT0800 - 22-Sep-23.pdf
Earnings_Transcripts/Competitor Earnings From Jul 2023/300628 - Event 厦门亿联网络技术股份有限公司 2023年半年度业绩说明会 2

### Show the object table

Confirm that the results display the PDF document in your Cloud Storage bucket.

In [46]:
%%bigquery --project $PROJECT_ID

SELECT *
FROM `docai_demo.object_table`;

Query is running:   0%|          |

Downloading:   0%|          |

,uri,generation,content_type,size,md5_hash,updated,metadata
0,gs://cisco_factset/Earnings_Transcripts/Compet...,1744156909135671,application/pdf,158857,b60dfe2f13c3cea6e10e674b81ca9a17,2025-04-09 00:01:49.171000+00:00,[]
1,gs://cisco_factset/Earnings_Transcripts/Compet...,1744156909135635,application/pdf,283953,f8367bd0f3be327962cb1e0ed85647aa,2025-04-09 00:01:49.171000+00:00,[]
2,gs://cisco_factset/Earnings_Transcripts/Compet...,1744156909215409,application/pdf,278896,51b3431a13ce1aae9d48ee7ce320f6e3,2025-04-09 00:01:49.256000+00:00,[]
3,gs://cisco_factset/Earnings_Transcripts/Compet...,1744156910260704,application/pdf,303694,10f9a2dbe5e2f2d3db179312c0334d64,2025-04-09 00:01:50.299000+00:00,[]
4,gs://cisco_factset/Earnings_Transcripts/Compet...,1744156909215307,application/pdf,290762,5c640fe543be8de51043efe38373e90b,2025-04-09 00:01:49.256000+00:00,[]
5,gs://cisco_factset/Earnings_Transcripts/Compet...,1744156910296454,application/pdf,313613,2a0eb3884661f761a048e2a13ca20842,2025-04-09 00:01:50.343000+00:00,[]
6,gs://cisco_factset/Earnings_Transcripts/Compet...,1744156910142549,application/pdf,302194,aea8aaa79f63d9d134b58d6f07f35b11,2025-04-09 00:01:50.186000+00:00,[]
7,gs://cisco_factset/Earnings_Transcripts/Compet...,1744156910139443,application/pdf,167251,05c7c67495b9c7c55ccccee806cc5179,2025-04-09 00:01:50.167000+00:00,[]
8,gs://cisco_factset/Earnings_Transcripts/Compet...,1744156910232989,application/pdf,295226,2f112b787394537d3892f26d08bc9b08,2025-04-09 00:01:50.278000+00:00,[]
9,gs://cisco_factset/Earnings_Transcripts/Compet...,1744156910164897,application/pdf,289176,bb6f7ee4af080ab1d00b99e33706f103,2025-04-09 00:01:50.199000+00:00,[]


## Use BQML and Document AI to parse documents

### Create a Layout Parser Processor in Document AI

[Create a new processor](https://cloud.google.com/document-ai/docs/create-processor#documentai_fetch_processor_types-python) in Document AI with the type `LAYOUT_PARSER_PROCESSOR`.

In [48]:
from google.api_core.client_options import ClientOptions
from google.cloud import documentai

location = "us"
processor_display_name = "layout_parser_processor"
processor_type = "LAYOUT_PARSER_PROCESSOR"


def create_processor_sample(
    PROJECT_ID: str, location: str, processor_display_name: str, processor_type: str
) -> None:
    opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

    client = documentai.DocumentProcessorServiceClient(client_options=opts)

    # The full resource name of the location
    parent = client.common_location_path(PROJECT_ID, location)

    # Create a processor
    processor = client.create_processor(
        parent=parent,
        processor=documentai.Processor(
            display_name=processor_display_name, type_=processor_type
        ),
    )

    # Return the processor ID needed for creating a BigQuery connection
    return processor.name.split("/")[-1]


# Call this function to create the processor and return its ID
processor_id = create_processor_sample(
    PROJECT_ID, location, processor_display_name, processor_type
)

### Create a remote model in BigQuery that connects with your Document AI Layout Parser Processor

This one-time setup step allows BigQuery to reference the Document AI Processor you just created.

**Note:** If if you receive an 400 GET error "permission denied for document processor", you may need to wait a minute for permissions to propagate from earlier steps.

In [49]:
query = f"""
CREATE OR REPLACE MODEL `docai_demo.layout_parser`
REMOTE WITH CONNECTION `us.demo_conn`
OPTIONS(remote_service_type="CLOUD_AI_DOCUMENT_V1", document_processor="{processor_id}")
"""

query_job = client.query(query)  # API request
query_job.result()  # Waits for the query to complete

print("Remote model docai_demo.layout_parser created or replaced successfully.")

Remote model docai_demo.layout_parser created or replaced successfully.


### Process the document using BigQuery ML

Use the [`ML.PROCESS_DOCUMENT` function](https://cloud.google.com/bigquery/docs/process-document) from BigQuery to call your Document AI processor and pass through the PDF. This uses the Layout Parser configuration and chunks your document.

**Note:** this may take a minute or so to complete.

In [50]:
%%bigquery --project $PROJECT_ID --location us

CREATE or REPLACE TABLE docai_demo.demo_result AS (
  SELECT * FROM ML.PROCESS_DOCUMENT(
  MODEL docai_demo.layout_parser,
  TABLE docai_demo.object_table,
  PROCESS_OPTIONS => (JSON '{"layout_config": {"chunking_config": {"chunk_size": 250}}}')
  )
);

Query is running:   0%|          |

""


### Parse the JSON results returned to BigQuery

The `ML.PROCESS_DOCUMENT` function parses the PDF from Cloud Storage and returns a JSON blob to BigQuery. In this step, we'll parse the JSON, extract document chunks and metadata, and return it to a new BigQuery table.

In [51]:
%%bigquery --project $PROJECT_ID --location us

CREATE OR REPLACE TABLE docai_demo.demo_result_parsed AS (
SELECT
  uri,
  JSON_EXTRACT_SCALAR(json , '$.chunkId') AS id,
  JSON_EXTRACT_SCALAR(json , '$.content') AS content,
  JSON_EXTRACT_SCALAR(json , '$.pageFooters[0].text') AS page_footers_text,
  JSON_EXTRACT_SCALAR(json , '$.pageSpan.pageStart') AS page_span_start,
  JSON_EXTRACT_SCALAR(json , '$.pageSpan.pageEnd') AS page_span_end
FROM docai_demo.demo_result, UNNEST(JSON_EXTRACT_ARRAY(ml_process_document_result.chunkedDocument.chunks, '$')) json
);

Query is running:   0%|          |

""


### Display the parsed document chunks

Show a preview of the parsed results and metadata.

In [52]:
%%bigquery --project $PROJECT_ID --location us

SELECT *
FROM docai_demo.demo_result_parsed
ORDER BY id
LIMIT 5;

Query is running:   0%|          |

Downloading:   0%|          |

,uri,id,content,page_footers_text,page_span_start,page_span_end
0,gs://cisco_factset/Earnings_Transcripts/Compet...,c1,30-Jul-2024 NEC Corp. (6701.JP) Q1 2025 Earnin...,FACTSET:callstreet 1-877-FACTSET www.callstree...,1,2
1,gs://cisco_factset/Earnings_Transcripts/Compet...,c1,05-Nov-2024 Super Q1 2025 Earnings Call Micro ...,FACTSET:callstreet 1-877-FACTSET www.callstree...,1,3
2,gs://cisco_factset/Earnings_Transcripts/Compet...,c1,13-Nov-2024 Cisco Q1 2025 Earnings Call System...,FACTSET:callstreet 1-877-FACTSET www.callstree...,1,3
3,gs://cisco_factset/Earnings_Transcripts/Compet...,c1,"20-Nov-2024 Palo Alto Networks, Inc. (PANW) Q1...",FACTSET:callstreet 1-877-FACTSET www.callstree...,1,3
4,gs://cisco_factset/Earnings_Transcripts/Compet...,c1,"27-Feb-2025 HP, Inc. (HPQ) Q1 2025 Earnings Ca...",FACTSET:callstreet 1-877-FACTSET www.callstree...,1,2


## Connect to Vertex AI embedding generation and Gemini access

### Connect to a text embedding model

[Create a remote model](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create-remote-model) allowing BigQuery access to a text embedding model hosted in Vertex AI.

In [53]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE MODEL `docai_demo.embedding_model`
REMOTE WITH CONNECTION `us.demo_conn` OPTIONS(endpoint="text-embedding-005")

Query is running:   0%|          |

""


### Generate embeddings

Use the [`ML.GENERATE_EMBEDDING` function](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-generate-embedding) in BigQuery to generate embeddings for all text chunks in the document.

In [54]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE TABLE `docai_demo.embeddings` AS
SELECT * FROM ML.GENERATE_EMBEDDING(
  MODEL `docai_demo.embedding_model`,
  TABLE `docai_demo.demo_result_parsed`
);

Query is running:   0%|          |

""


### Connect to a Gemini LLM endpoint

Create a remote model allowing BigQuery access to a Gemini foundation model hosted in Vertex AI.

In [55]:
%%bigquery --project $PROJECT_ID

CREATE OR REPLACE MODEL `docai_demo.gemini_flash` REMOTE
WITH CONNECTION `us.demo_conn` OPTIONS(endpoint="gemini-2.0-flash")

Query is running:   0%|          |

""


## Run vector search, return results, and pass them to Gemini for text generation

### Sample BigQuery vector search

Run a sample BigQuery vector search against your chunks. This query takes your text input, creates an embedding using the `ML.GENERATE_EMBEDDING` function, and then passes the embedding through to the [`VECTOR_SEARCH` function](https://cloud.google.com/bigquery/docs/reference/standard-sql/search_functions#vector_search). The results are the top ten chunks that are most semantically related to your input.

In the search query below, the input text asks "Did the typical family net worth increase? If so, by how much?"

In [56]:
%%bigquery --project $PROJECT_ID

SELECT query.query, base.uri, base.id, base.content, distance
    FROM
      VECTOR_SEARCH( TABLE `docai_demo.embeddings`,
        'ml_generate_embedding_result',
        (
        SELECT
          ml_generate_embedding_result,
          content AS query
        FROM
          ML.GENERATE_EMBEDDING( MODEL `docai_demo.embedding_model`,
            ( SELECT 'What is the outlook of the CEO on future performance for Palo Alto?' AS content)
          )
        ),
        top_k => 5,
        OPTIONS => '{"fraction_lists_to_search": 0.005}')
ORDER BY distance DESC;

Query is running:   0%|          |

Downloading:   0%|          |

,query,uri,id,content,distance
0,What is the outlook of the CEO on future perfo...,gs://cisco_factset/Earnings_Transcripts/Compet...,c20,FACTSET:callstreet 1-877-FACTSET www.callstree...,0.760930
1,What is the outlook of the CEO on future perfo...,gs://cisco_factset/Earnings_Transcripts/Compet...,c5,## Nikesh Arora\n\nChairman & Chief Executive ...,0.744920
2,What is the outlook of the CEO on future perfo...,gs://cisco_factset/Earnings_Transcripts/Compet...,c60,# Nikesh Arora\n\nChairman & Chief Executive O...,0.740794
3,What is the outlook of the CEO on future perfo...,gs://cisco_factset/Earnings_Transcripts/Compet...,c40,Hey guys. Thanks for the question. I think you...,0.730425
4,What is the outlook of the CEO on future perfo...,gs://cisco_factset/Earnings_Transcripts/Compet...,c3,## Walter H. Pritchard\n\nSenior Vice Presiden...,0.730278


## Generate text augmented by vector search results

This step builds upon the prior one - but instead of simply returning the top text chunks, it calls the `ML.GENERATE_TEXT` function to summarize them alongside the question we input.

In this query you:
* **Retrieve** the closest chunks semantically using the `VECTOR_SEARCH` function (this is what was done in the prior query)
* **Augment** the Gemini LLM with this knowledge
* **Generate** a succinct answer using the `ML.GENERATE_TEXT` function

In [23]:
%%bigquery --project $PROJECT_ID

SELECT
  ml_generate_text_llm_result AS generated,'\n',
  prompt -- Commented out, but please feel free to uncomment if you would like to see the full context passed to the Gemini model
FROM
  ML.GENERATE_TEXT( MODEL `docai_demo.gemini_flash`,
    (
    SELECT CONCAT(   """You are an expert on Cisco Systems' business answering questions.
        Use all retrieved documents to answer the questions.
        If multiple sources have conflicting answers, provide all answers given and relevant information.
        Structure your response in the following format:
        1. Answer: (Provide a details anser to the question)
        2. Supporting Research: (List the documents that were used to formulate your answer and the similarity score/distance)
        3. Quotes: (For each Supporting Document, provide exact quotes used to support your answer.Show name of each document and the quote used)
        4. Additional Information: (Provides quotes from all Retrieved documents that are relevant to the question, but was not directly used in the answer)
        5. If you are not sure about the answer, say 'I am not sure. Please rephrase the question and/or expand the search space'""",'\n',
    STRING_AGG(FORMAT("context: %s and reference: %s and similarity score: %s", base.content, base.uri, CAST(distance AS STRING)),',\n'),
      FORMAT("User question: %s", max(query.query))
    ) AS prompt,
    FROM
      VECTOR_SEARCH( TABLE
        `docai_demo.embeddings`,
        'ml_generate_embedding_result',
        (
        SELECT
          ml_generate_embedding_result,
          content AS query
        FROM
          ML.GENERATE_EMBEDDING( MODEL `docai_demo.embedding_model`,
            (
            SELECT
              "What is the impact of QRadar?" AS content
            )
          )
        ),
        top_k => 10,
        distance_type => "COSINE",
        OPTIONS => '{"fraction_lists_to_search": 0.005}')
      ),
      STRUCT(1000 AS max_output_tokens, TRUE AS flatten_json_output)
  );

Query is running:   0%|          |

Downloading:   0%|          |

,generated,f0_,prompt
0,1. **Answer:**\n * Palo Alto Networks (P...,\n,You are an expert on Cisco Systems' business a...


### Sample questions to try out:

Here are a list of a few other questions to spark your imagination. Feel free to try your own too!
* Did the amount of debt families own on their home increase between 2019 and 2022?
* Did younger or older families see their net worth increase more?
* How much did the median family income change between 2018 and 2021?

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial by uncommenting the below:

In [5]:
 # Deletes the BigQuery assets and Google Cloud Storage bucket

 !bq rm -r -f $PROJECT_ID:docai_demo
 !bq rm --connection --project_id=$PROJECT_ID --location=us demo_conn
 !gsutil rm -r gs://{bucket_name}


 # Deletes the Document AI processor
 def delete_processor_sample(
     PROJECT_ID: str, location: str, processor_id: str
 ) -> None:
     """Deletes a processor."""

     opts = ClientOptions(api_endpoint=f"{location}-documentai.googleapis.com")

     client = documentai.DocumentProcessorServiceClient(client_options=opts)

     # The full resource name of the processor
     name = f"projects/{PROJECT_ID}/locations/{location}/processors/{processor_id}"

     try:
         client.delete_processor(name=name)
         print(f"Processor {processor_id} deleted successfully.")
     except Exception as e:
         print(f"Error deleting processor: {e}")


 # Call this function to delete the processor
 delete_processor_sample(PROJECT_ID, location, processor_id)

BucketNotFoundException: 404 gs://{bucket_name} bucket does not exist.


NameError: name 'location' is not defined

# Wrap up

This notebook demonstrates an example of how to achieve a basic end-to-end retrieval-augmented generation pipeline using BigQuery. It integrates BigQuery ML functions like `ML.PROCESS_DOCUMENT` to call Document AI and parse PDFs, `ML.GENERATE_EMBEDDING` to generate embeddings on text chunks and input queries, and `ML.GENERATE_TEXT` to provide a concise answer. It also uses the `VECTOR_SEARCH` function to identify similar text (using embeddings) in BigQuery using familiar SQL syntax.

To continue learn more, check out our documentation on [BigQuery ML](https://cloud.google.com/bigquery/docs/bqml-introduction) and [BigQuery Vector Search](https://cloud.google.com/bigquery/docs/vector-search).